# core

> Segmentation step state management helpers

In [ ]:
#| default_exp routes.core

In [ ]:
#| export
from typing import List, Dict, Any, NamedTuple

from cjm_fasthtml_card_stack.core.models import CardStackState
from cjm_fasthtml_card_stack.core.constants import DEFAULT_VISIBLE_COUNT, DEFAULT_CARD_WIDTH

from cjm_fasthtml_interactions.core.state_store import get_session_id
from cjm_workflow_state.history import push_history
from cjm_workflow_state.state_store import SQLiteWorkflowStateStore

from cjm_transcript_segmentation.models import (
    TextSegment, SegmentationStepState
)

# Type alias for state store (duck-typed, accepts any implementation with get_state/update_state)
WorkflowStateStore = SQLiteWorkflowStateStore

# Debug flag for state update tracing (set False in production)
DEBUG_SEG_STATE = False

# Default max history depth when not configured
DEFAULT_MAX_HISTORY_DEPTH = 50

## State Management Helpers

In [ ]:
#| export
class SegContext(NamedTuple):
    """Common segmentation state values loaded by handlers."""
    segment_dicts: List[Dict[str, Any]]  # Serialized working segments
    focused_index: int  # Currently focused segment index
    visible_count: int  # Number of visible cards in viewport
    is_auto_mode: bool  # Whether card count is in auto-adjust mode
    card_width: int  # Card stack width in rem
    history: list  # Undo history stack

In [ ]:
#| export
def _to_segments(
    segment_dicts: List[Dict[str, Any]]  # Serialized segment dictionaries
) -> List[TextSegment]:  # Deserialized TextSegment objects
    """Convert segment dictionaries to TextSegment objects."""
    return [TextSegment.from_dict(s) for s in segment_dicts]

In [ ]:
#| export
def _get_seg_state(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    session_id: str  # Session identifier string
) -> SegmentationStepState:  # Segmentation step state dictionary
    """Get the segmentation step state from the workflow state store."""
    workflow_state = state_store.get_state(workflow_id, session_id)
    step_states = workflow_state.get("step_states", {})
    return step_states.get("segmentation", {})

In [ ]:
#| export
def _get_selection_state(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    session_id: str  # Session identifier string
) -> Dict[str, Any]:  # Selection step state dictionary
    """Get the selection step state (Phase 1) from the workflow state store."""
    workflow_state = state_store.get_state(workflow_id, session_id)
    step_states = workflow_state.get("step_states", {})
    return step_states.get("selection", {})

In [ ]:
#| export
def _build_card_stack_state(
    ctx: SegContext,  # Loaded segmentation context
    active_mode: str = None,  # Active interaction mode (e.g. "split")
) -> CardStackState:  # Card stack state for library functions
    """Build a CardStackState from segmentation context for library calls."""
    return CardStackState(
        focused_index=ctx.focused_index,
        visible_count=ctx.visible_count,
        card_width=ctx.card_width,
        active_mode=active_mode,
    )

In [ ]:
#| export
def _load_seg_context(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    session_id: str  # Session identifier string
) -> SegContext:  # Common segmentation state values
    """Load commonly-needed segmentation state values in a single call."""
    seg_state = _get_seg_state(state_store, workflow_id, session_id)
    return SegContext(
        segment_dicts=seg_state.get("segments", []),
        focused_index=seg_state.get("focused_index", 0),
        visible_count=seg_state.get("visible_count", DEFAULT_VISIBLE_COUNT),
        is_auto_mode=seg_state.get("is_auto_mode", False),
        card_width=seg_state.get("card_width", DEFAULT_CARD_WIDTH),
        history=seg_state.get("history", []),
    )

In [ ]:
#| export
def _update_seg_state(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    session_id: str,  # Session identifier string
    segments: List[Dict[str, Any]] = None,  # Updated segments (None = don't change)
    initial_segments: List[Dict[str, Any]] = None,  # Initial segments for reset (None = don't change)
    focused_index: int = None,  # Updated focused index (None = don't change)
    is_initialized: bool = None,  # Initialization flag (None = don't change)
    history: List[Dict[str, Any]] = None,  # Updated history (None = don't change)
    visible_count: int = None,  # Visible card count (None = don't change)
    is_auto_mode: bool = None,  # Auto-adjust mode flag (None = don't change)
    card_width: int = None,  # Card stack width in rem (None = don't change)
) -> None:
    """Update the segmentation step state in the workflow state store."""
    workflow_state = state_store.get_state(workflow_id, session_id)

    if DEBUG_SEG_STATE:
        print(f"[SEG_STATE] _update_seg_state called")
        print(f"[SEG_STATE] BEFORE: step_states keys = {list(workflow_state.get('step_states', {}).keys())}")
        selection_state = workflow_state.get('step_states', {}).get('selection', {})
        print(f"[SEG_STATE] BEFORE: selection.selected_sources count = {len(selection_state.get('selected_sources', []))}")

    step_states = workflow_state.get("step_states", {})
    seg_state = step_states.get("segmentation", {})
    
    # Update only provided fields
    if segments is not None:
        seg_state["segments"] = segments
    if initial_segments is not None:
        seg_state["initial_segments"] = initial_segments
    if focused_index is not None:
        seg_state["focused_index"] = focused_index
    if is_initialized is not None:
        seg_state["is_initialized"] = is_initialized
    if history is not None:
        seg_state["history"] = history
    if visible_count is not None:
        seg_state["visible_count"] = visible_count
    if is_auto_mode is not None:
        seg_state["is_auto_mode"] = is_auto_mode
    if card_width is not None:
        seg_state["card_width"] = card_width
    
    step_states["segmentation"] = seg_state
    workflow_state["step_states"] = step_states

    if DEBUG_SEG_STATE:
        print(f"[SEG_STATE] AFTER (before update_state): step_states keys = {list(workflow_state.get('step_states', {}).keys())}")
        selection_state_after = workflow_state.get('step_states', {}).get('selection', {})
        print(f"[SEG_STATE] AFTER (before update_state): selection.selected_sources count = {len(selection_state_after.get('selected_sources', []))}")
        print(f"[SEG_STATE] Calling update_state with full workflow_state...")

    state_store.update_state(workflow_id, session_id, workflow_state)

    if DEBUG_SEG_STATE:
        # Re-read state to verify what was actually stored
        verify_state = state_store.get_state(workflow_id, session_id)
        print(f"[SEG_STATE] VERIFY (after update_state): step_states keys = {list(verify_state.get('step_states', {}).keys())}")
        verify_selection = verify_state.get('step_states', {}).get('selection', {})
        print(f"[SEG_STATE] VERIFY (after update_state): selection.selected_sources count = {len(verify_selection.get('selected_sources', []))}")

In [ ]:
#| export
def _push_history(
    state_store: WorkflowStateStore,  # The workflow state store
    workflow_id: str,  # The workflow identifier
    session_id: str,  # Session identifier string
    current_segments: List[Dict[str, Any]],  # Current segments to snapshot
    focused_index: int,  # Current focused index to snapshot
    max_history_depth: int = DEFAULT_MAX_HISTORY_DEPTH,  # Maximum history stack depth
) -> int:  # New history depth after push
    """Push current state to history stack before making changes."""
    seg_state = _get_seg_state(state_store, workflow_id, session_id)
    history = seg_state.get("history", [])
    
    snapshot = {"segments": current_segments, "focused_index": focused_index}
    new_history = push_history(history, snapshot, max_history_depth)
    _update_seg_state(state_store, workflow_id, session_id, history=new_history)
    return len(new_history)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()